In [3]:
#main_wiki_htmlとinfo_fir_searchを作成します
import pickle,os
from bs4 import BeautifulSoup
from create_info_from_html import create_categories,encode_filename
#まず漫画について処理を行うよ
pickle_year_lsit = os.listdir("../manga_wiki_html_data/")
info_for_search_dict = {}
categories_list = []
actor_list = []
info_list = []
genre_list = [] #一応追加しておく

for pickle_year in pickle_year_lsit:
    with open(f"../manga_wiki_html_data/{pickle_year}","rb") as f:
        year =int(pickle_year.replace(".pickle",""))
        # if year <2004:continue
        load_data = pickle.load(f)
    for title,res in load_data.items():
        # print(res)
        soup = BeautifulSoup(res, 'html.parser')
        page_title = soup.find("title").get_text().replace(" - Wikipedia","") #こちらの方がより効率が良さそう
        if page_title in info_for_search_dict:
            #既に同一のwikiページのページがdictに存在している場合、つまりここを通るのは派生作品
            print(f"{year}:{page_title}からの派生作品：{title}")
            _is_same_title_type_media = False
            for index,manga in enumerate(info_for_search_dict[page_title]["media_dict"]["manga"]):
                if manga["title"] == title:
                    info_for_search_dict[page_title]["media_dict"]["manga"][index]["year"] += f",{year}" #文字列としてyearを追加していく
                    _is_same_title_type_media = True
                    break

            if not _is_same_title_type_media:info_for_search_dict[page_title]["media_dict"]["manga"].append({"title":title,"year":str(year),"amazon_img":""}) #派生作品のタイトルと年だけを追加する
        else:
            #新規に追加していく場合
            print(f"{year}:新規追加：{title}")
            info_for_search_dict[page_title] = {"categories":[],"genre":[],"actor_dict":{},"infobox_dict":{},"infobox_list":[],"media_dict":{"manga":[],"anime":[],"novel":[]}}
            categories,actor_dict,return_table_list,return_table_dict,genre = create_categories(soup)
            info_for_search_dict[page_title]["categories"] = categories
            info_for_search_dict[page_title]["genre"] = genre
            info_for_search_dict[page_title]["actor_dict"] = actor_dict
            info_for_search_dict[page_title]["infobox_dict"] =return_table_dict
            info_for_search_dict[page_title]["infobox_list"] =return_table_list

            info_for_search_dict[page_title]["media_dict"]["manga"].append({"title":title,"year":str(year),"amazon_img":""}) #新規に情報をリストとして追加する,派生作品はこの中に入っていく！
            
            #新規ページの場合、htmlを追加する
            os.makedirs(f"../../data_for_search/main_wiki_html/{year}", exist_ok=True) #ディレクトリ作成
            with open(f"../../data_for_search/main_wiki_html/{year}/{encode_filename(page_title)}.pickle","wb") as f:
                pickle.dump(res,f)

            #カテゴリや名前についての情報をリストに追加する
            categories_list = list(set(categories_list + categories))
            actor_list = list(set(actor_list + list(actor_dict.keys())))
            info_list = list(set(info_list + return_table_list))
            genre_list = list(set(genre_list + genre))

with open("../../data_for_search/info_for_search_dict.pickle","wb") as f:
    pickle.dump(info_for_search_dict,f)

with open("../../data_for_search/categories_list.pickle","wb") as f:
    pickle.dump(categories_list,f)
with open("../../data_for_search/actor_list.pickle","wb") as f:
    pickle.dump(actor_list,f)
with open("../../data_for_search/info_list.pickle","wb") as f:
    pickle.dump(info_list,f)
with open("../../data_for_search/genre_list.pickle","wb") as f:
    pickle.dump(genre_list,f)

1930:新規追加：クックとプッケ
1930:新規追加：スピード太郎
1930:新規追加：タンタンのコンゴ探険
1930:新規追加：ブロンディ (漫画)
1931:新規追加：タンタン アメリカへ
1931:新規追加：のらくろ
1932:新規追加：ファラオの葉巻
1933:新規追加：ファニーズ・オンパレード
1933:新規追加：冒険ダン吉
1934:新規追加：青い蓮
1934:新規追加：タンクタンクロー
1934:新規追加：フラッシュ・ゴードン
1935:新規追加：かけた耳
1935:新規追加：リトル・ルル
1936:新規追加：ザ・ファントム
1936:新規追加：フクちゃん
1937:新規追加：黒い島のひみつ
1938:新規追加：アクション・コミックス
1938:新規追加：オトカル王の杖
1939:新規追加：スーパーマン
1940:新規追加：金のはさみのカニ
1940:新規追加：グリーンランタン
1940:新規追加：ジャスティス・ソサエティ・オブ・アメリカ
1940:新規追加：仲よし手帖
1940:新規追加：バットマン (コミック)
1940:新規追加：フラッシュ (DCコミックス)
1941:新規追加：アクアマン
1941:新規追加：グリーンアロー
1941:新規追加：ふしぎな流れ星
1941:新規追加：プラスチックマン
1941:新規追加：翼賛一家大和さん
1941:新規追加：ワンダーウーマン
1942:新規追加：なぞのユニコーン号
1946:新規追加：サザエさん
1946:新規追加：マァチャンの日記帳
1946:新規追加：ヤネウラ3ちゃん
1946:新規追加：ロストワールド (漫画)
1947:新規追加：火星博士
1947:新規追加：新宝島
1947:新規追加：バット君
1947:新規追加：ムーミンまんがシリーズ
1947:新規追加：リル・フォークス


KeyboardInterrupt: 

In [2]:
#次に小説についても同様の処理を行う
#小説について、page_title.replace(" (小説)","") in manga_page_title_dict or page_title.replace(" (小説)"," (漫画)") in manga_page_title_dict
#という処理を行った見たのですが、昔の作品などで特に顕著だったのが全く関係ない作品（例えば太宰治のやつとか）と同じ名前があるってのがあるので、とりあえず小説についてはreplaceをしない方針でいきます
with open("../../data_for_search/info_for_search_dict.pickle","rb") as f:
    info_for_search_dict =pickle.load(f)

pickle_year_lsit = os.listdir("../novel_wiki_html_data/")
for pickle_year in pickle_year_lsit:
    with open(f"../novel_wiki_html_data/{pickle_year}","rb") as f:
        year =int(pickle_year.replace(".pickle",""))
        # if year <2020:continue
        load_data = pickle.load(f)
    for title,res in load_data.items():
        # print(res)
        soup = BeautifulSoup(res, 'html.parser')
        page_title = soup.find("title").get_text().replace(" - Wikipedia","") #こちらの方がより効率が良さそう
        if page_title in info_for_search_dict:
            #既に同一のwikiページのページがdictに存在している場合、つまりここを通るのは派生作品
            print(f"{year}:{page_title}からの派生作品：{title}")
            _is_same_title_type_media = False
            for index,novel in enumerate(info_for_search_dict[page_title]["media_dict"]["novel"]):
                if novel["title"] == title:
                    info_for_search_dict[page_title]["media_dict"]["novel"][index]["year"] += f",{year}" #文字列としてyearを追加していく
                    _is_same_title_type_media = True
                    break

            if not _is_same_title_type_media:info_for_search_dict[page_title]["media_dict"]["novel"].append({"title":title,"year":str(year),"amazon_img":""}) #派生作品のタイトルと年だけを追加する
        
        else:
            #新規に追加していく場合
            print(f"{year}:新規追加：{title}")
            info_for_search_dict[page_title] = {"categories":[],"genre":[],"actor_dict":{},"infobox_dict":{},"infobox_list":[],"media_dict":{"manga":[],"anime":[],"novel":[]}}
            categories,actor_dict,return_table_list,return_table_dict,genre = create_categories(soup)
            info_for_search_dict[page_title]["categories"] = categories
            info_for_search_dict[page_title]["genre"] = genre
            info_for_search_dict[page_title]["actor_dict"] = actor_dict
            info_for_search_dict[page_title]["infobox_dict"] =return_table_dict
            info_for_search_dict[page_title]["infobox_list"] =return_table_list

            info_for_search_dict[page_title]["media_dict"]["novel"].append({"title":title,"year":str(year),"amazon_img":""}) #新規に情報をリストとして追加する,派生作品はこの中に入っていく！

            #新規ページの場合、htmlを追加する
            os.makedirs(f"../../data_for_search/main_wiki_html/{year}", exist_ok=True)
            with open(f"../../data_for_search/main_wiki_html/{year}/{encode_filename(page_title)}.pickle","wb") as f:
                pickle.dump(res,f)

            #カテゴリや名前についての情報をリストに追加する
            categories_list = list(set(categories_list + categories))
            actor_list = list(set(actor_list + list(actor_dict.keys())))
            info_list = list(set(info_list + return_table_list))
            genre_list = list(set(genre_list + genre))

            
with open("../../data_for_search/info_for_search_dict.pickle","wb") as f:
    pickle.dump(info_for_search_dict,f)
    
with open("../../data_for_search/categories_list.pickle","wb") as f:
    pickle.dump(categories_list,f)
with open("../../data_for_search/actor_list.pickle","wb") as f:
    pickle.dump(actor_list,f)
with open("../../data_for_search/info_list.pickle","wb") as f:
    pickle.dump(info_list,f)
with open("../../data_for_search/genre_list.pickle","wb") as f:
    pickle.dump(genre_list,f)

1900:新規追加：オズの魔法使い
1900:新規追加：海底軍艦
1900:新規追加：神さまの話
1900:新規追加：高野聖 (小説)
1900:新規追加：小間使の日記
1900:新規追加：シスター・キャリー
1900:新規追加：社会百面相
1900:新規追加：バッソンピエール元帥の体験
1901:新規追加：月世界最初の人間
1901:新規追加：バスカヴィル家の犬
1901:新規追加：ブッデンブローク家の人々
1901:新規追加：ポゾール王の冒険
1902:新規追加：犬物語
1902:新規追加：猿の手
1902:新規追加：サンタクロースの冒険
1902:新規追加：酒中日記
1902:新規追加：砂の妖精
1902:新規追加：チャンドス卿の手紙
1902:新規追加：鳩の翼
1902:新規追加：マーヘヴナのクフーリン
1902:新規追加：闇の奥
1903:新規追加：空き家の冒険
1903:新規追加：踊る人形
1903:新規追加：グラディーヴァ
1903:新規追加：自動車を待つ間
1903:新規追加：ジャン・クリストフ
1903:新規追加：探検奨学金
1903:新規追加：乳姉妹 (小説)
1903:新規追加：トーニオ・クレーガー
1903:新規追加：ノーウッドの建築業者
1903:新規追加：魔風恋風
1903:新規追加：ミネハハ (小説)
1903:新規追加：野性の呼び声
1904:新規追加：青柳のはなし
1904:新規追加：かけひき
1904:新規追加：金縁の鼻眼鏡
1904:新規追加：愚者の渡しの防御
1904:新規追加：孤独な自転車乗り
1904:新規追加：三人の学生
1904:新規追加：食人鬼 (小説)
1904:新規追加：スリークウォーター失踪
1904:新規追加：世界の支配者
1904:新規追加：僧坊荘園
1904:新規追加：第二の汚点
1904:新規追加：犯人は二人
1904:新規追加：プライオリ学校
1904:新規追加：ブラック・ピーター
1904:新規追加：葬られた秘密
1904:新規追加：緑の館
1904:新規追加：六つのナポレオン
1904:新規追加：女夫波
1905:新規追加：一夜
1905:新規追加：薤露行
1905:新規追加：琴のそら音
1905:新規追加：車輪の下
1905:新規追加：真紅の花の歌
1905:新規追加：正義の四人
1905:新規

In [3]:
#次にアニメについても同様の処理を行う

with open("../../data_for_search/info_for_search_dict.pickle","rb") as f:
    info_for_search_dict =pickle.load(f)

pickle_year_lsit = os.listdir("../anime_wiki_html_data/")
for pickle_year in pickle_year_lsit:
    with open(f"../anime_wiki_html_data/{pickle_year}","rb") as f:
        year =int(pickle_year.replace(".pickle",""))
        # if year <2020:continue
        load_data = pickle.load(f)
    for title,res in load_data.items():
        # print(res)
        soup = BeautifulSoup(res, 'html.parser')
        page_title = soup.find("title").get_text().replace(" - Wikipedia","") #こちらの方がより効率が良さそう
        if page_title in info_for_search_dict:
            #既に同一のwikiページのページがdictに存在している場合、つまりここを通るのは派生作品
            print(f"{year}:{page_title}からの派生作品：{title}")
            _is_same_title_type_media = False
            for index,anime in enumerate(info_for_search_dict[page_title]["media_dict"]["anime"]):
                if anime["title"] == title:
                    info_for_search_dict[page_title]["media_dict"]["anime"][index]["year"] += f",{year}" #文字列としてyearを追加していく
                    _is_same_title_type_media = True
                    break

            if not _is_same_title_type_media:info_for_search_dict[page_title]["media_dict"]["anime"].append({"title":title,"year":str(year),"amazon_img":""}) #派生作品のタイトルと年だけを追加する
        
        elif " (アニメ)"in page_title and  page_title.replace(" (アニメ)","") in info_for_search_dict:
            print(f"{year}:page_titleから( アニメ)を取ったら元々入っていた{page_title}と同じになりました:{title}")
            page_title = page_title.replace(" (アニメ)","")
            #この場合はページが完全に別なので、infoboxなどの情報を追加して入れる必要がある
            categories,actor_dict,return_table_list,return_table_dict,genre = create_categories(soup)
            info_for_search_dict[page_title]["infobox_dict"].update(return_table_dict) #dictを結合
            info_for_search_dict[page_title]["infobox_list"] = list(set(info_for_search_dict[page_title]["infobox_list"]+return_table_list))
            info_for_search_dict[page_title]["categories"] = list(set(info_for_search_dict[page_title]["categories"] + categories))
            info_for_search_dict[page_title]["genre"] = list(set(info_for_search_dict[page_title]["genre"] + genre))
            info_for_search_dict[page_title]["media_dict"]["anime"].append({"title":title,"year":year,"amazon_img":""}) 

            #カテゴリや名前についての情報をリストに追加する
            categories_list = list(set(categories_list + categories))
            actor_list = list(set(actor_list + list(actor_dict.keys())))
            info_list = list(set(info_list + return_table_list))
            genre_list = list(set(genre_list + genre))

            
        else:
            #新規に追加していく場合
            print(f"{year}:新規追加：{title}")
            info_for_search_dict[page_title] = {"categories":[],"genre":[],"actor_dict":{},"infobox_dict":{},"infobox_list":[],"media_dict":{"manga":[],"anime":[],"novel":[]}}
            categories,actor_dict,return_table_list,return_table_dict,genre = create_categories(soup)
            info_for_search_dict[page_title]["categories"] = categories
            info_for_search_dict[page_title]["genre"] = genre
            info_for_search_dict[page_title]["actor_dict"] = actor_dict
            info_for_search_dict[page_title]["infobox_dict"] =return_table_dict
            info_for_search_dict[page_title]["infobox_list"] =return_table_list
            info_for_search_dict[page_title]["media_dict"]["anime"].append({"title":title,"year":str(year),"amazon_img":""}) #新規に情報をリストとして追加する,派生作品はこの中に入っていく！
            #新規ページの場合、htmlを追加する
            os.makedirs(f"../../data_for_search/main_wiki_html/{year}", exist_ok=True)
            with open(f"../../data_for_search/main_wiki_html/{year}/{encode_filename(page_title)}.pickle","wb") as f:
                pickle.dump(res,f)
            
            #カテゴリや名前についての情報をリストに追加する
            categories_list = list(set(categories_list + categories))
            actor_list = list(set(actor_list + list(actor_dict.keys())))
            info_list = list(set(info_list + return_table_list))
            genre_list = list(set(genre_list + genre))

with open("../../data_for_search/info_for_search_dict.pickle","wb") as f:
    pickle.dump(info_for_search_dict,f)
    
with open("../../data_for_search/categories_list.pickle","wb") as f:
    pickle.dump(categories_list,f)
with open("../../data_for_search/actor_list.pickle","wb") as f:
    pickle.dump(actor_list,f)
with open("../../data_for_search/info_list.pickle","wb") as f:
    pickle.dump(info_list,f)
with open("../../data_for_search/genre_list.pickle","wb") as f:
    pickle.dump(genre_list,f)

1957:新規追加：ウッディー・ウッドペッカー・ショー
1957:新規追加：チンチンの冒険 (テレビアニメ)
1957:新規追加：ディズニークラシック短編集
1957:新規追加：漫画ニュース
1958:新規追加：珍犬ハックル
1958:新規追加：つよいぞラフティ
1958:新規追加：フィリックス・ザ・キャット (アニメ)
1958:新規追加：もぐらのアバンチュール
1958:新規追加：ゆかいなボゾ
1959:新規追加：オギーとダディー
1959:新規追加：クラッチ・カーゴ
1959:新規追加：テレビ坊やの冒険
1959:新規追加：バッキーとペピート
1959:新規追加：早射ちマック
1959:新規追加：ロッキーとブルウィンクルの大冒険
1960:新規追加：新しい動画 3つのはなし
1960:新規追加：王とオデイ
1960:新規追加：カレイジャス・キャット
1960:新規追加：原始家族フリントストーン
1960:新規追加：探偵スカット
1960:新規追加：バッグス・バニー・ショー
1961:新規追加：インスタントヒストリー
1961:新規追加：ドラ猫大将
1962:新規追加：宇宙家族ジェットソン
1962:おとぎマンガカレンダーからの派生作品：おとぎマンガカレンダー
1962:新規追加：キャプテン・ゼロ
1962:新規追加：リッピーとハーディー
1963:8マンからの派生作品：エイトマン
1963:新規追加：狼少年ケン
1963:新規追加：ゴリラのゴンちゃん
1963:シスコン王子からの派生作品：進めシスコン
1963:仙人部落からの派生作品：仙人部落
1963:新規追加：タキシード・ペンギン
1963:新規追加：鉄人28号 (テレビアニメ第1作)
1963:新規追加：鉄腕アトム (アニメ第1作)
1963:フィリックス・ザ・キャット (アニメ)からの派生作品：とびだせフィリックス
1963:新規追加：ピーコック劇場 (テレビ番組)
1963:新規追加：ファニーカンパニー (テレビアニメ)
1963:新規追加：フィリックス・ザ・キャット
1963:新規追加：マイティ・ハーキュリー
1964:新規追加：ウルトラわんちゃん
1964:新規追加：科学少年J.Q
1964:新規追加：少年忍者風のフジ丸
1964:新規追加：0戦はやと
1964:新規追加：トムとジェリー
196

In [20]:
#漫画、アニメ、小説のどれがあったのかを示すための追加情報media_typesを追加します
import pickle
with open("../../data_for_search/info_for_search_dict.pickle","rb") as f:
    info_for_search_dict =pickle.load(f)

buff_info_for_search_dict = {}
for page_title,data in info_for_search_dict.items():
    categories = data["categories"]
    infobox_dict = data["infobox_dict"]
    media_dict = data["media_dict"]
    _ismanga = True if any(["の漫画" in info for info in categories]) or any(["漫画" in info for info in infobox_dict]) or media_dict["manga"]!=[] else False
    _isanime = True if any(["のテレビアニメ"   in info or "アニメ"   in info or "OVA" in info for info in categories]) or any(["アニメ" in info for info in infobox_dict]) or media_dict["anime"]!=[] else False
    _isnovel = True if any(["の小説" in info for info in categories]) or any(["小説" in info for info in infobox_dict]) or media_dict["novel"]!=[] else False
    # print(f"{_ismanga},{_isanime},{_isnovel}")
    # if all([_ismanga,_isanime,_isnovel]):print(page_title)
    data["media_types"] = {"manga":_ismanga,"anime":_isanime,"novel":_isnovel} #media_typesという情報を新たに追加する
    buff_info_for_search_dict[page_title] = data
   
with open("../../data_for_search/info_for_search_dict.pickle","wb") as f:
    pickle.dump(buff_info_for_search_dict,f)

In [1]:
#次にamazonからimg画像をスクレイピングする,漫画>小説>アニメ　の順でスクレイピングしたほうがよいのでは？
import pickle
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException

# wait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

# selenium 4.0 ↑
from selenium.webdriver.common.by import By
from time import sleep
import pickle,re
import unicodedata

chrome_options = Options()
# chrome_options.add_argument('--headless')
driver = webdriver.Chrome(options=chrome_options) 

from get_amazon_url import get_amazon_link_and_img
with open("../../data_for_search/info_for_search_dict.pickle","rb") as f:
    info_for_search_dict =pickle.load(f)

#ここからスクレイピング＆取得したurlとimgを保存する作業
count = 1
for page_title,data in info_for_search_dict.items():
    media_type = None
    if data["media_types"]["manga"]:
        media_type="manga"
    elif data["media_types"]["novel"]:
        media_type="novel"
    elif data["media_types"]["anime"]:
        media_type="anime"
    
    for index,media_data in enumerate(data["media_dict"][media_type]):
        if media_data["amazon_img"] =="":
            print(f"imgありません！！  {page_title}")
            amazon_url,amazon_img = get_amazon_link_and_img(driver,page_title,media_type)
            info_for_search_dict[page_title]["media_dict"][media_type][index]["amazon_img"] = amazon_img 
            info_for_search_dict[page_title]["media_dict"][media_type][index]["amazon_url"] = amazon_url
            count+=1
            print(amazon_url,amazon_img)
            break
            
        else:break
    
    
    if count%20==0:
        with open(r"../../data_for_search/info_for_search_dict.pickle","wb") as f:
            pickle.dump(info_for_search_dict,f)
        driver.quit() #chromeでout of memoryになってしまうことへの対策、何回かごとに再起動させる
        driver = webdriver.Chrome(options=chrome_options) 
        print("保存しました")
        count = 1
    

imgありません！！  魔の刻
https://www.amazon.co.jp/%E3%83%8E%E3%83%BC%E3%83%96%E3%83%A9%E3%83%B3%E3%83%89%E5%93%81-00540079-%E9%AD%94%E3%81%AE%E5%88%BB-%E5%8C%97%E6%B3%89%E5%84%AA%E5%AD%90%E8%91%97-%E5%B8%AF%E4%BB%98/dp/B0CVVKHQJF/ https://m.media-amazon.com/images/I/81M4Q8Q9ftL._AC_UY218_.jpg
imgありません！！  マンハッタンの奇譚クラブ
取得に失敗しました！
None None
imgありません！！  三毛猫ホームズの恐怖館
https://www.amazon.co.jp/%E3%83%8E%E3%83%BC%E3%83%96%E3%83%A9%E3%83%B3%E3%83%89%E5%93%81-00570370-%E3%80%90%E5%B8%AF%E4%BB%98%E3%81%8D%E3%80%91%E4%B8%89%E6%AF%9B%E7%8C%AB%E3%83%9B%E3%83%BC%E3%83%A0%E3%82%BA%E3%81%AE%E7%8B%82%E6%AD%BB%E6%9B%B2%E3%83%BB%E4%B8%89%E6%AF%9B%E7%8C%AB%E3%83%9B%E3%83%BC%E3%83%A0%E3%82%BA%E3%81%AE%E9%A7%86%E8%90%BD%E3%81%A1%E3%83%BB%E4%B8%89%E6%AF%9B%E7%8C%AB%E3%83%9B%E3%83%BC%E3%83%A0%E3%82%BA%E3%81%AE%E6%81%90%E6%80%96%E9%A4%A8%EF%BC%883%E5%86%8A%E3%82%BB%E3%83%83%E3%83%88%EF%BC%89%EF%BC%8F%E8%B5%A4%E5%B7%9D%E6%AC%A1%E9%83%8E/dp/B0CTYH73K3/ https://m.media-amazon.com/images/I/51gXVY214VL._AC_UY218_.jpg
imgありません

In [ ]:
#保存したhtmlを呼び出すのに必須なyearをinfoに追加していなかったのでそれを追加するようのコード
#このコードを動作させるとinfo_for_search_dict[page_title]に page_year　が追加されます. htmlを参照するときには page_year/page_title.pickle　を参照すればよいので楽！
import pickle
with open("../../data_for_search/info_for_search_dict.pickle","rb") as f:
    info_for_search_dict =pickle.load(f)

import os
from create_info_from_html import decode_filename
years = os.listdir("../../data_for_search/main_wiki_html/")
page_title_and_year_dict = {}
for year in years:
    page_titles =  os.listdir(f"../../data_for_search/main_wiki_html/{year}/")
    page_titles  = [decode_filename(page_title) for page_title in page_titles]
    for page_title in page_titles:
        page_title_and_year_dict[page_title.replace(".pickle","")] = int(year)
        # print(f"{page_title}  {page_title_and_year_dict[page_title]}")

for page_title,data in info_for_search_dict.items():
    info_for_search_dict[page_title]["page_year"] =page_title_and_year_dict[page_title] #page_yearというものを新規に追加する

with open("../../data_for_search/info_for_search_dict.pickle","wb") as f:
    pickle.dump(info_for_search_dict,f)